In [225]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [226]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [227]:
bs=52 # 52 - Jeremey, 20 - default

In [228]:
bptt= 70 #70 - Jeremey, 35 - default

In [229]:
df_train.head()

,label,text,words,tokens
18413,2,I've just got back from watching this delightf...,"[ \n , xxbos, xxfld, 1, i, 've, just, got, bac...","[41, 42, 43, 40, 13, 161, 57, 210, 162, 51, 16..."
43883,2,"For a first film, this shows a great deal of v...","[ \n , xxbos, xxfld, 1, xxmaj, for, a, first, ...","[41, 42, 43, 40, 2, 24, 7, 106, 27, 5, 14, 298..."
45342,2,I am fully aware that no film adaptation of a ...,"[ \n , xxbos, xxfld, 1, i, am, fully, aware, t...","[41, 42, 43, 40, 13, 264, 1390, 1811, 15, 74, ..."
6454,2,I'm always very cautious when people refer to ...,"[ \n , xxbos, xxfld, 1, i, 'm, always, very, c...","[41, 42, 43, 40, 13, 167, 236, 70, 15406, 69, ..."
13703,2,I started to watch this movie but gave up beca...,"[ \n , xxbos, xxfld, 1, i, started, to, watch,...","[41, 42, 43, 40, 13, 677, 9, 127, 14, 25, 26, ..."


In [230]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [231]:
n_batch=df_train.shape[0]//bs
padlen=640
padding_idx=1

In [232]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [233]:
df_train['n_tok'].describe()

count    67500.000000
mean       305.567867
std        225.163517
min         15.000000
25%        166.000000
50%        229.000000
75%        371.000000
max       3455.000000
Name: n_tok, dtype: float64

In [234]:
df_train.sort_values(by='n_tok', inplace=True)

In [235]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [236]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [237]:
for i in range(0,n_batch):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640
[41. 42. 43. 40. ...  1.  1.  1.  1.]
2
640


In [238]:
df_train.shape

(67500, 5)

In [239]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(2    45002
 1    11252
 0    11246
 Name: label, dtype: int64, 2    4998
 0    1254
 1    1248
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [240]:
df_train_big=df_train.copy()

In [241]:
df_train=df_train.loc[df_train['label']<2]

In [242]:
df_train.shape

(22498, 5)

In [243]:
np.ones(3).astype(int)

array([1, 1, 1])

In [244]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [245]:
padlen=400

In [246]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [247]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [248]:
for xb,yb,xlen in dltrain:
    break

In [249]:
xb, yb, xlen

(tensor([[ 41,  42,  43,  ...,   1,   1,   1],
         [ 41,  42,  43,  ...,   1,   1,   1],
         [ 41,  42,  43,  ...,   1,   1,   1],
         ...,
         [ 41,  42,  43,  ...,   1,   1,   1],
         [ 41,  42,  43,  ...,   1,   1,   1],
         [ 41,  42,  43,  ...,   3,  67, 238]]),
 tensor([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
         1, 0, 0, 1]),
 tensor([319, 257, 201, 246, 380,  69, 170, 321, 380, 166, 190, 198, 355, 236,
         834, 132, 377, 160, 435, 394, 732, 231, 129, 152, 246, 880,  72, 277,
         918, 208,  71, 216, 371, 245, 295, 159, 537,  45, 170, 178, 150, 114,
         239, 661,  64, 144, 143, 165, 408, 122, 149, 508]))

## Model Architecture

In [250]:
from torch import nn
import torch
from torch.autograd import Variable

In [251]:
bs,bptt

(52, 70)

In [288]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=1150 #650
n_layers=3
dropout=0.5
wd=1e-7
bidirectional=False
dropout_e=0.05
dropout_o=0.5
n_out=2

In [253]:
?torch.nn.utils.rnn.pack_padded_sequence

In [254]:
?torch.nn.utils.rnn.pad_packed_sequence

In [255]:
xb.shape,xlen.shape

(torch.Size([52, 400]), torch.Size([52]))

In [351]:
df_train['label'].value_counts()

1    11252
0    11246
Name: label, dtype: int64

In [337]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out=n_out
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        self.gen_hidden()
        self.create_architecture()
        if pretrain_mtx is not None:
            print ("initializing")
            self.initialize_glove()
        self.criterion=nn.BCEWithLogitsLoss()
        
    def create_architecture(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape n_batch * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        self.fc=nn.Linear(self.n_hidden,self.n_out)
        #self.log_softmax=nn.LogSoftmax()
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data=torch.Tensor(self.pretrain_mtx)
    
    def gen_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
        
    def forward(self,Xb,Yb,Xb_lengths):
        #import pdb
        #pdb.set_trace()
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        
        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        Xb = torch.nn.utils.rnn.pack_padded_sequence(Xb,Xb_lengths,batch_first=True,enforce_sorted=False)
        out,new_hidden=self.lstm(embs,self.hidden)

        
        # after lstm call
        #Xb,_=torch.nn.utils.rnn.pad_packed_sequence(Xb,batch_first=True)
        out=self.dropout_op(out)

        # Wrap the hidden state in a new tensor without the gradients
        self.hidden=(Variable(new_hidden[0].data,requires_grad=False).to(self.device),\
                     Variable(new_hidden[1].data,requires_grad=False).to(self.device))
        
        #preds=self.fc(out.contiguous().view(out.size(0)*out.size(1), out.size(2)))

         # output [:,-1,:] is the same as hn[-1]
        out=self.fc (out[:,-1,:])
        #out=self.log_softmax(out)
    

        preds=out.view(-1)
        # Mask out the loss function
        if 1==0:
            mask=(actual != self.padding_idx)
            actual=actual[mask]
            preds=preds[mask]
        loss=self.criterion(preds,Yb.float())
        
        return preds, loss  

In [359]:
def accuracy_binomial(preds,actual):
    correct=torch.round(preds).long()==yb
    return correct.float().sum()/len(correct)

In [339]:
device="cpu"

## Load pretrained model weights

In [340]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [341]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [342]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 1150, 3, False, 52, 'cpu', 0.05, 0.5, 0.5, 2)

In [343]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing


In [344]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.05)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 1150, num_layers=3, batch_first=True, dropout=0.5)
  (dropout_op): Dropout(p=0.5)
  (fc): Linear(in_features=1150, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [345]:
model_sentiment.forward(xb,yb,xlen)

(tensor([0.0269, 0.0338, 0.0267, 0.0400, 0.0240, 0.0308, 0.0292, 0.0241, 0.0167,
         0.0296, 0.0271, 0.0222, 0.0139, 0.0171, 0.0482, 0.0335, 0.0321, 0.0398,
         0.0174, 0.0334, 0.0300, 0.0228, 0.0088, 0.0245, 0.0262, 0.0403, 0.0286,
         0.0394, 0.0232, 0.0411, 0.0281, 0.0180, 0.0248, 0.0354, 0.0105, 0.0314,
         0.0217, 0.0390, 0.0177, 0.0193, 0.0224, 0.0439, 0.0365, 0.0295, 0.0333,
         0.0313, 0.0283, 0.0230, 0.0319, 0.0221, 0.0233, 0.0403],
        grad_fn=<ViewBackward>),
 tensor(0.6919, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [346]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [352]:
preds

tensor([0.0268, 0.0335, 0.0155, 0.0261, 0.0327, 0.0330, 0.0367, 0.0285, 0.0443,
        0.0238, 0.0284, 0.0281, 0.0346, 0.0272, 0.0230, 0.0447, 0.0298, 0.0385,
        0.0331, 0.0332, 0.0099, 0.0226, 0.0275, 0.0358, 0.0436, 0.0351, 0.0299,
        0.0202, 0.0373, 0.0289, 0.0256, 0.0536, 0.0451, 0.0151, 0.0269, 0.0401,
        0.0368, 0.0234, 0.0216, 0.0378, 0.0229, 0.0226, 0.0322, 0.0201, 0.0152,
        0.0369, 0.0448, 0.0343, 0.0195, 0.0244, 0.0399, 0.0092],
       grad_fn=<ViewBackward>)

In [353]:
yb

tensor([0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
        1, 0, 0, 1])

In [358]:
torch.round(preds).long()==yb

tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 1, 0], dtype=torch.uint8)

In [361]:
accuracy_binomial(preds,yb)

tensor(0.4423)

In [362]:
from sklearn.metrics import roc_auc_score

In [364]:
roc_auc_score(yb,preds)

0.5262368815592204

## Learner

In [366]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1))
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
            if 1==0:
                for p in self.model.parameters():
                    p.data.add_(self.lr, p.grad.data)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i=0,0,0
        self.model.gen_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/k
        epoch_acc=epoch_acc/k
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [374]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=1e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [373]:
device="cuda:1"

In [371]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,500,0.25)

In [375]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model=model.to(device)

initializing


In [377]:
model_sentiment.freeze_embedding()

In [378]:
model_sentiment.encoder.weight.requires_grad

False

In [ ]:
learner.run_epochs(dltrain,dlvalid,1)

In [86]:
learner.device=device
learner.model=learner.model.to(device)
model.device=device
learner.model.device=device
learner.model.hidden=(learner.model.hidden[0].cuda(1),learner.model.hidden[1].cuda(1))
model.hidden=(model.hidden[0].cuda(1),model.hidden[1].cuda(1))


In [92]:
def set_gpu_recursive(var, gpu_id):
    for key in var:
        if isinstance(var[key], dict):
            var[key] = set_gpu_recursive(var[key], gpu_id)
        else:
            try:
                var[key] = var[key].cuda(gpu_id)
            except:
                pass
    return var



In [100]:
optimizer.state = set_gpu_recursive(optimizer.state, 1)
learner.optimizer.state = set_gpu_recursive(learner.optimizer.state, 1)

In [103]:
learner.run_epochs(train_tokens,valid_tokens,1)

Batch:500 4.215455349445343  0.2790599019825459
Batch:1000 4.210862180948258  0.2788794076293707
Batch:1500 4.212145571072896  0.2783448839088281
Batch:2000 4.211311246871948  0.278226111009717
Batch:2500 4.210199825286865  0.27802792423963546
Batch:3000 4.209871447563171  0.27792235642671587
Batch:3500 4.2090058584213255  0.27796406232459203
Batch:4000 4.207844992935658  0.2780102457590401
Batch:4500 4.207080701986949  0.27803181937999194
Batch:5000 4.206981196069718  0.2780740782529116
Batch:5500 4.205610530679876  0.27823452775315805
Epoch:0 Loss:4.20557110817913
Batch:500 3.911064175128937  0.30575276267528534
Epoch:0 Loss:4.20557110817913 Accuracy:0.27822952477167406 Loss:3.910363855164207 Accuracy:0.30540491747513915


In [104]:
np.exp(3.9), np.exp(3.91)

(49.40244910553017, 49.89895197340787)

In [106]:
torch.save(model.state_dict(),f'{PATH}/inter/model_awd_lstm_state_dict_cuda_1')
torch.save(optimizer.state_dict(),f'{PATH}/inter/learner_awd_lstm_state_dict_cuda_1')


In [107]:
torch.save (model,f'{PATH}/inter/model_awd_lstm_cuda_1')


In [108]:
torch.save (optimizer,f'{PATH}/inter/optimizer_awd_lstm_cuda_1')


In [109]:
torch.save (learner,f'{PATH}/inter/learner_awd_lstm_cuda_1')

## Transfer Learning to predict RNN